In [ ]:
#i "nuget:/home/pi/localNuget"

In [ ]:
#r "nuget:piTop"
#r "nuget:piTop.FoundationKit"

In [ ]:
using PiTopMakerArchitecture.Foundation;
using PiTopMakerArchitecture.Foundation.Components;
using PiTopMakerArchitecture.Foundation.Sensors;

In [ ]:
var plate = new Plate();

In [ ]:
var greenLed = plate.GetOrCreateDigitalDevice<Led>(DigitalPort.D0);
greenLed.DisplayProperties.Add(new NamedCssColor("green"));

In [ ]:
var amberLed = plate.GetOrCreateDigitalDevice<Led>(DigitalPort.D1);
amberLed.DisplayProperties.Add(new NamedCssColor("gold"));

In [ ]:
var redLed = plate.GetOrCreateDigitalDevice<Led>(DigitalPort.D2);
redLed.DisplayProperties.Add(new NamedCssColor("red"));

In [ ]:
greenLed.Off();
amberLed.Off();
redLed.Off();

In [ ]:
display(plate, JsonFormatter.MimeType);

In [ ]:
plate

In [ ]:
var pot = plate.GetOrCreateAnalogueDevice<Potentiometer>(AnaloguePort.A0);

In [ ]:
pot

In [ ]:
plate

In [ ]:
greenLed.On();
greenLed

In [ ]:
amberLed.On();
amberLed

In [ ]:
redLed.On();
redLed

In [ ]:
greenLed.Toggle();
greenLed

In [ ]:
var ultraSound = plate.GetOrCreateDigitalDevice<UltrasonicSensor>(DigitalPort.D3);

In [ ]:
ultraSound

In [ ]:
plate

In [ ]:
var distances = new List<double>(){0};

async Task readDistance(int count = 10){   
    for (var i = 0; i < count; i++){        
        if(distances.Count > 5){
            distances.RemoveAt(0);
        }
        distances.Add(ultraSound.Distance);
        await Task.Delay(500);
    }
}

In [ ]:
#!js 

if (typeof (notebookScope.interval) !== 'undefined') {
    clearInterval(notebookScope.interval);
}        

notebookScope.plot = (sgvSelector, variableName) => {   
    let dtree_require = require.config({
        paths: {
            d3: "https://d3js.org/d3.v5.min"
        }
    });
    dtree_require(["d3"], function (d3) {       
        let svg = d3.
            select(sgvSelector);
        svg.selectAll("defs").remove();
        svg.selectAll("g").remove();

        let defs = svg.append("defs");

        let filter = defs.append("filter").attr("id", "gooeyCodeFilter");

        filter.append("feGaussianBlur")
            .attr("in", "SourceGraphic")
            .attr("stdDeviation", "10")
            .attr("color-interpolation-filters", "sRGB")
            .attr("result", "blur");

        filter.append("feColorMatrix")
            .attr("in", "blur")
            .attr("mode", "matrix")
            .attr("values", "1 0 0 0 0  0 1 0 0 0  0 0 1 0 0  0 0 0 19 -9")
            .attr("result", "gooey");

        let container = d3
            .select(sgvSelector)
            .append("g")
            .style("filter", "url(#gooeyCodeFilter)");

        updateD3Rendering = () => interactive.csharp.getVariable(variableName)
            .then(data => {
                container
                    .selectAll(".points")
                    .data(data, (d, i) => i)
                    .join(
                        enter => enter.append("circle")
                            .attr("class", "points")
                            .attr("cy", 80)
                            .attr("cx", (d, i) => ((i + 1) * 50))
                            .transition()
                            .duration(2000)
                            .style("fill", d => d3.interpolateTurbo(d / 50))
                            .ease(d3.easeElasticOut.period(1.00))
                            .attr("r", d => Math.max(0, d)),
                        update => update
                            .transition()
                            .duration(2000)                            
                            .style("fill", d => d3.interpolateTurbo(d / 50))
                            .attr("r", d => Math.max(0, d)),
                        exit => exit
                            .transition()
                            .duration(1000)                            
                            .attr("r", 0 )
                            .remove());
            });
        notebookScope.interval = setInterval(() => updateD3Rendering(), 1000);
    });
}

In [ ]:
#!html
<svg id="dataPlot1" width="100%"></svg>

#!js
notebookScope.plot("svg#dataPlot1", "distances");

In [ ]:
await readDistance(20);

In [ ]:
distances